In [0]:
%pip install geopy

# Augment data with geolocation

The geopy library is sed to add lat/long to the policy location for better visualization of the data in the end result dashboard for the Claims Investigators.

Telematics data include mileage and driving habits and provide a wealth of information that is useful for insurance companies. <br />
This is useful across multiple use cases and serves as data to calculate new rates for policyholders, often on a rolling basis. Cleansing this data and enriching it with geo-spatial features of latitude ad longitude will accelerate processing and can provide visual pointers on dashboards eg. the route of the trip and where it encountered the accident.

<!-- Collect usage data (view). Remove it to disable collection or disable tracker during installation. View README for more details.  -->
<img width="1px" src="https://ppxrzfxige.execute-api.us-west-2.amazonaws.com/v1/analytics?category=lakehouse&org_id=4325389025886535&notebook=%2F01-Data-Ingestion%2F01.2-Policy-Location&demo_name=lakehouse-fsi-smart-claims&event=VIEW&path=%2F_dbdemos%2Flakehouse%2Flakehouse-fsi-smart-claims%2F01-Data-Ingestion%2F01.2-Policy-Location&version=1">

In [0]:
import dlt
import geopy
import pandas as pd
from pyspark.sql.functions import col, lit, concat, pandas_udf
from typing import Iterator
import random

# Function to get latitude and longitude from geocoding result
def geocode(geolocator, address):
    try:
      #Skip the API call for faster demo (remove this line for ream)
      return pd.Series({'latitude':  random.uniform(-90, 90), 'longitude': random.uniform(-180, 180)})
      location = geolocator.geocode(address)
      if location:
          return pd.Series({'latitude': location.latitude, 'longitude': location.longitude})
    except Exception as e:
      print(f"error getting lat/long: {e}")
    return pd.Series({'latitude': None, 'longitude': None})
      
@pandas_udf("latitude float, longitude float")
def get_lat_long(batch_iter: Iterator[pd.Series]) -> Iterator[pd.DataFrame]:
  #ctx = ssl.create_default_context(cafile=certifi.where())
  #geopy.geocoders.options.default_ssl_context = ctx
  geolocator = geopy.Nominatim(user_agent="claim_lat_long", timeout=5, scheme='https')
  for address in batch_iter:
    yield address.apply(lambda x: geocode(geolocator, x))

In [0]:
@dlt.table(comment="claims with geolocation latitude/longitude")
def claim_policy_telematics():
  t = dlt.read("telematics")
  claim = dlt.read("claim_policy").where("address is not null")
  return (claim.withColumn("lat_long", get_lat_long(col("address")))
               .join(t, on="chassis_no"))

In [0]:
claim_policy_telematics_df = spark.table("dbdemos_fsi_smart_claims.claim_policy_telematics").toPandas()
claim_policy_telematics_df.head(10)

CHASSIS_NO  policy_no  ... telematics_latitude telematics_longitude
0  WBAHT1102H5H73209  102107383  ...           40.764183           -73.889391
1  WDDHF5FB0GB262628  102107384  ...           40.804515           -73.925425
2  MHFGX8EM3H0226134  102107386  ...           40.764183           -73.889391
3  WBAJB3100HWA04668  102107400  ...           40.804515           -73.925425
4  LSYYBBCB9DC007653  102107406  ...           40.811540           -73.965922
5  ZAMVL45F0G0175159  102107409  ...           40.811540           -73.965922
6  KL1CD6B14DC514178  102107413  ...           40.804515           -73.925425
7  JN1CC11CX8T013824  102107427  ...           40.811540           -73.965922
8  JTHBE5D21H5029202  102107449  ...           40.811540           -73.965922
9  WVWFM2AU7HW123833  102107450  ...           40.811540           -73.965922

[10 rows x 44 columns]